# PXTEXTMINING - API Testing 


In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings
import datetime 
import requests
import os

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

In [2]:
from friendsfamilytest.params import *
from friendsfamilytest.utils import *
from friendsfamilytest.data import *

In [3]:
current_directory = os.getcwd()
print(f'cwd: {current_directory}')
try:
    directory_path = "../friendsfamilytest/data/"
    file_list = os.listdir(directory_path)
    print(f'🚀 {file_list}')
except Exception as e:
    print(f'❌: {e}')

cwd: /Users/janduplessis/code/janduplessis883/friends-and-family-test-analysis/notebooks
🚀 ['processed.csv', '.DS_Store', 'new.csv', 'data.csv']


In [4]:
text_data = [
              { 'comment_id': '1', # The comment_id values in each dict must be unique.
                'comment_text': 'This is the first comment. Nurse was great.',
                'question_type': 'what_good' },
              { 'comment_id': '2',
                'comment_text': 'This is the second comment. The ward was freezing.',
                'question_type': 'could_improve' },
              { 'comment_id': '3',
                'comment_text': '',  # This comment is an empty string.
                'question_type': 'nonspecific' }
            ]

### Generate `text_data` list from current database

In [5]:
data = pd.read_csv('../friendsfamilytest/data/data.csv')

In [6]:
data.head(2)

,time,rating,free_text,do_better,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound
0,13/07/2023 15:28,Extremely likely,very friendly caring staff treated me with res...,NaN,13,0,5.0,admiration,0.588930,positive,0.866406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13/07/2023 15:29,Extremely likely,so good so confidential,NaN,4,0,5.0,admiration,0.910946,positive,0.925523,NaN,0.7,0.6,0.0,0.484,0.516,0.4927


In [7]:
def make_api_data(pandas_series, question_type='could_improve'):
    text_data = []

    for index, row in pandas_series.iteritems():
        # Check if the row is NaN or empty ('')
        if pd.isna(row) or row == '':
            comment_text = ''
        else:
            comment_text = row

        item_dict = {
            'comment_id': str(index), # The comment_id values in each dict must be unique.
            'comment_text': comment_text,
            'question_type': question_type
        }

        text_data.append(item_dict)
        
    return text_data

In [8]:
text_data = make_api_data(data['do_better'])

In [12]:
text_data

[{'comment_id': '0', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '1', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '2',
  'comment_text': 'no it was the best',
  'question_type': 'could_improve'},
 {'comment_id': '3',
  'comment_text': 'the receptionist was very unkind',
  'question_type': 'could_improve'},
 {'comment_id': '4',
  'comment_text': 'no i was happy with everything',
  'question_type': 'could_improve'},
 {'comment_id': '5', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '6',
  'comment_text': 'no it is already perfect',
  'question_type': 'could_improve'},
 {'comment_id': '7', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '8', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '9', 'comment_text': '', 'question_type': 'could_improve'},
 {'comment_id': '10',
  'comment_text': 'i think it is getting better the ladies at the reception can be so rude it woul

## Making API Call - pxtextmining 

In [16]:
import requests
import os
import time

url = "https://connect.strategyunitwm.nhs.uk/text_api/"
api_key = os.getenv('PXTEXTMINING_API_KEY')
params_dict = {'code': api_key, 'target': 'm'}

try:
    response = requests.post(url, params=params_dict, json=text_data)

    if response.status_code == 202:
        results_url = response.text
        print(f"URL for results is {results_url}")

        while True:
            results_response = requests.get(results_url)
            if results_response.status_code == 200:
                final_labels = results_response.json()
                break
            elif results_response.status_code == 202:
                print('Results not ready yet. Trying again in 300 seconds...')
                time.sleep(300)
            else:
                print(f'Error fetching results: Status code {results_response.status_code}')
                print('Details:', results_response.text)
                break

    elif response.status_code == 400:
        print("Bad Request - The request was invalid or cannot be otherwise served.")
        print('Details:', response.text)

    elif response.status_code == 401:
        print("Unauthorized - Authentication credentials were missing or incorrect.")
        print('Details:', response.text)

    elif response.status_code == 403:
        print("Forbidden - The request is understood, but it has been refused or access is not allowed.")
        print('Details:', response.text)

    elif response.status_code == 404:
        print("Not Found - The URI requested is invalid or the resource requested does not exist.")
        print('Details:', response.text)

    else:
        print(f"Unexpected Error: {response.status_code}")
        print('Details:', response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Unexpected Error: 405
Details: {"detail":"Method Not Allowed"}
